In [1]:
import os
import shutil
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from dataset.flame import FlameSatelite
from torchvision.transforms import transforms

In [7]:
df_annot = pd.read_csv("../../data/flame_satellite/annotations.csv")

In [8]:
df_annot['class'] = df_annot['class'].map({'Fire' : 1})

In [4]:
dataset_path = "../../data/flame_satellite"
images_path = dataset_path+"/images"
label_path = dataset_path+"/YOLO/labels"

In [10]:
grouped = df_annot.groupby("filename")

In [12]:
for filename, group in grouped:
    img_path = os.path.join(dataset_path+"/images", filename)
    if not os.path.exists(img_path):
        continue

    img = Image.open(img_path)
    img_w, img_h = img.size

    label_list = []
    for _, row in group.iterrows():
        x_center = (row["xmin"] + row["xmax"]) / 2 / img_w
        y_center = (row["ymin"] + row["ymax"]) / 2 / img_h
        width = (row["xmax"] - row["xmin"]) / img_w
        height = (row["ymax"] - row["ymin"]) / img_h

        yolo_label = f"{row['class']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        label_list.append(yolo_label)

    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(dataset_path+"/YOLO/labels", label_filename)

    with open(label_path, "w") as f:
        f.write("\n".join(label_list))

In [2]:
# for file in os.listdir(dataset_path+"/YOLO/images/train"):
#     shutil.move(dataset_path+f"/YOLO/images/train/{file}", f"{images_path}")

In [5]:
image_files = os.listdir(images_path)
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

for file in train_files:
    shutil.move(f"{images_path}/{file}", dataset_path+"/YOLO/images/train")
    if os.path.exists(f"{label_path}/{file.replace('.jpg', '.txt')}"):
        shutil.move(f"{label_path}/{file.replace('.jpg', '.txt')}", dataset_path+"/YOLO/labels/train")

for file in val_files:
    shutil.move(f"{images_path}/{file}", dataset_path+"/YOLO/images/val")
    if os.path.exists(f"{label_path}/{file.replace('.jpg', '.txt')}"):
        shutil.move(f"{label_path}/{file.replace('.jpg', '.txt')}", dataset_path+"/YOLO/labels/val")
